## Libraries

In [ ]:
import numpy as np
import mediapipe as mp
import cv2
from collections import defaultdict

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

## NO Half Reps

In [ ]:
## tool to count full squat reps also with the same mediapipe pre-trained model used for the triples extension

def no_half_reps(video):
    
    cap = cv2.VideoCapture(video)

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 800)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 600)
    
    # squat counter variables
    counter = 0 
    stage = "up" # used so the counter doesn't repeatedly counts + 1 in every moment the degree is < 90
    
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
          
            results = pose.process(image)
        
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            try:
                landmarks = results.pose_landmarks.landmark
                
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                        
                angle = calculate_angle(hip, knee, ankle)
                
                # squat counter logic
                if angle > 90 and stage == "up":
                    stage = "down"
                if angle < 90 and stage == "down":
                    stage = "up" 
                    counter +=1
                    #print(counter)
                           
            except:
                pass

            # set counter box
            cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
            
            cv2.putText(image, 'REPS', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                     )               
        
            cv2.imshow('NO Half Reps!', image)
    
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    
        cap.release()
        cv2.destroyAllWindows()

    return

no_half_reps(1)

## 1-Rep Max Calculator

In [ ]:
## decided to play around to estimate 1-rep max for exercises according to number of repetions x weight at maximum effort

def rep_max_calculator(weight, reps): 
    percentages = [1, 0.97, 0.94, 0.92, 0.89, 0.86, 0.83, 0.81, 0.78, 0.75, 0.73, 0.71, 0.70, 0.68, 0.67]
    return weight / percentages[reps-1]